In [ ]:
# ==== System info ====
!nvidia-smi -L || echo "CPU-only runtime"
!python --version

# ==== Install AutoGluon and Kaggle ====
!pip install -U pip
!pip install -U "autogluon>=1.0" kaggle

# ==== Ensure folders exist ====
import os
for d in ["data","artifacts"]:
    os.makedirs(d, exist_ok=True)
print("Setup complete. Remember: Edit ▸ Notebook settings ▸ uncheck 'Omit code cell output when saving'.")


In [ ]:
# ---- Multimodal tabular demo: add text feature ----
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

train = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
test  = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
label = 'class'

# Create a simple text feature by concatenating a few columns
def add_text_feature(df):
    df = df.copy()
    cols = [c for c in df.columns if c != label]
    use = [c for c in cols if df[c].dtype == 'object'][:3]  # take a few cat columns
    if len(use) >= 2:
        df['desc_text'] = df[use].astype(str).agg(' '.join, axis=1)
    else:
        df['desc_text'] = df[cols].astype(str).agg(' '.join, axis=1)
    return df

train_mm = add_text_feature(train)
test_mm  = add_text_feature(test)

predictor = TabularPredictor(label=label, path='ag_multimodal/').fit(
    train_data=train_mm,
    time_limit=900,
    presets='medium_quality_faster_train'
)

lb = predictor.leaderboard(test_mm, silent=True)
fi = predictor.feature_importance(test_mm)

lb_path = 'artifacts/multimodal_leaderboard.csv'
fi_path = 'artifacts/multimodal_feature_importance.csv'
lb.to_csv(lb_path, index=False)
fi.to_csv(fi_path)

print("Saved:", lb_path, fi_path)
lb.head(10), fi.head(10)
